In [ ]:
from langchain.agents import create_agent
import json
from langchain_openai import ChatOpenAI
from pprint import pprint
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

# define two models
basic_model = ChatOpenAI(
    model="llama3.2:3b",
    temperature=0,
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

advanced_model = ChatOpenAI(
    model="llama3.2:3b",
    temperature=0.5,
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    return handler(request.override(model=model))

# set model selector in middleware
agent = create_agent(
    model=basic_model,  # Default model
    tools=[],
    middleware=[dynamic_model_selection]
)

result = agent.invoke({"input": "What's the capital of France?", "messages": [{"role": "user", "content": "What's the capital of France?"}]})
print(result)

result_dict = dict(result)
for key, value in result_dict.items():
    print(key, value)




{'messages': [HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={}, id='d89018a4-64e8-4fb7-ba57-7c2d7043aec6'), AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 32, 'total_tokens': 40, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'llama3.2:3b', 'system_fingerprint': 'fp_ollama', 'id': 'chatcmpl-165', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--4b8f165a-77ea-4d4b-9dbc-6d7e1f5cfcb9-0', usage_metadata={'input_tokens': 32, 'output_tokens': 8, 'total_tokens': 40, 'input_token_details': {}, 'output_token_details': {}})]}
messages [HumanMessage(content="What's the capital of France?", additional_kwargs={}, response_metadata={}, id='d89018a4-64e8-4fb7-ba57-7c2d7043aec6'), AIMessage(content='The capital of France is Paris.', additional_kwargs={'refusal': None}, 